# Train SetFit Mood Model

This notebook trains a SetFit model for mood detection using the provided dataset. 
**Upload `mood_training_data.json` and `hard_mood_data.json` to the Files tab before running.**

In [ ]:
# 1. Install Dependencies
!pip install setfit datasets sentence-transformers

In [ ]:
# 2. Imports
import os
# Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"
import json
import shutil
from datasets import Dataset, concatenate_datasets
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss

In [ ]:
# 3. Load & Prepare Data
print("📚 Loading Data...")

base_data = []
hard_data = []

if os.path.exists("mood_training_data.json"):
    with open("mood_training_data.json", 'r') as f:
        base_data = json.load(f)
else:
    print("⚠️ Warning: mood_training_data.json not found.")

if os.path.exists("hard_mood_data.json"):
    with open("hard_mood_data.json", 'r') as f:
        hard_data = json.load(f)
else:
    print("⚠️ Warning: hard_mood_data.json not found.")

all_data = base_data + hard_data
texts = [d['text'] for d in all_data]
labels = [d['mood'] for d in all_data]

dataset = Dataset.from_dict({"text": texts, "label": labels})

# Few-Shot Sampling (100 per class)
print("✂️ Sampling 100 examples per class for efficient training...")
train_ds_list = []
test_ds_list = []
unique_labels = set(labels)

for label in unique_labels:
    subset = dataset.filter(lambda x: x["label"] == label)
    count = len(subset)
    # Use 100 for train, 20 for test (or less if not enough data)
    train_size = min(100, int(count * 0.8))
    test_size = min(20, int(count * 0.2))
    
    split = subset.train_test_split(train_size=train_size, test_size=test_size, seed=42)
    train_ds_list.append(split["train"])
    test_ds_list.append(split["test"])

train_ds = concatenate_datasets(train_ds_list)
test_ds = concatenate_datasets(test_ds_list)

print(f"   -> Train Size: {len(train_ds)}")
print(f"   -> Test Size:  {len(test_ds)}")

In [ ]:
# 4. Train Model
print("🚀 Training SetFit Model (all-MiniLM-L6-v2)...")
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # Good balance for quality
    num_epochs=1,
    column_mapping={"text": "text", "label": "label"}
)

trainer.train()
metrics = trainer.evaluate()
print(f"\n📊 Evaluation Results: {metrics}")

In [ ]:
# 5. Save & Export
print("💾 Saving Model...")
output_dir = "setfit_mood_model"
model.save_pretrained(output_dir)

print("📦 Zipping Model for Export...")
shutil.make_archive(output_dir, 'zip', output_dir)
print(f"✅ Done! Please download '{output_dir}.zip' from the Files tab.")